In [44]:
import os
import cv2
import numpy as np
import mediapipe as mp
import uuid
import requests
import imutils
import time

In [37]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [43]:
url = "http://192.168.107.216:8080/shot.jpg"
handno = 0
drowing = False
fingarid = [4,8,12,16,20]
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while True:
        # Load Images to mobile
        img_resp = requests.get(url)
        img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
        img = cv2.imdecode(img_arr, -1)
        frame = imutils.resize(img, width=800, height=1000)
        
        #Hand landmark
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(255,20,147), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(127,255,0), thickness=2, circle_radius=2),
                )
        # Detect Hand Position
        array_landmar = []
        if results.multi_hand_landmarks:
            myhand = results.multi_hand_landmarks[handno]
            for id ,m in enumerate(myhand.landmark):
                h,w,c = image.shape
                cx,cy = int(m.x * w) , int(m.y * h)
                array_landmar.append([id ,cx, cy])
                if drowing:
                    cv2.circle(image,(cx,cy),15,(255,0,255),cv2.FILLED)
        
        # Count Numbers
        if len(array_landmar) != 0:
            count = []
            if array_landmar[fingarid[0]][1] > array_landmar[fingarid[0] - 1][1]:
                count.append(1)
            else:
                count.append(0)

            for n in range(1 ,5):
                if array_landmar[fingarid[n]][2] < array_landmar[fingarid[n] - 2][2]:
                    count.append(1)
                else:
                    count.append(0)
            total_count = count.count(1)

            cv2.putText(image,str(total_count),(650,420),cv2.FONT_HERSHEY_PLAIN,8,(148,0,211),10)
      
        # Show Images
        cv2.imshow('Hand Tracking', image)
        
        # Close window using a keyword
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

        
cv2.destroyAllWindows()